# Imports

In [ ]:
%load_ext lab_black

import h5py
import os

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm

from typing import Dict, List, Optional, Tuple
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\TR"
files = []

## High Statistics Scans -- at M to K##
# files = [f"TR4_Ali_avg.h5"]
# files = [f"TR6_Ali_avg.h5"]  # -0.5 to 0.5 ps
# files = [
#     f"TR7_Ali_avg.h5"
# ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

## High Statistics Scans -- at G ##
# files = [f"TR9&TR11_Ali_avg.h5"]  # -0.5 to 0.5 ps
# files = [
#     f"TR12_Ali_avg_best.h5"
# ]  # -1 to 1 ps in steps of 100 fs, then out to 30 ps with variable steps

# ### February Comparison ###
ddir = r"E:\atully\arpes_data\2023_February\6eV\TR"
files = []
# files = ["TR3_Ali_avg.h5"]
# files = ["TR_001_1.h5"]
files = [
    "TR4_Ali_avg.h5"
]  # 2.6 eV center energy; -1 to 1 ps, same number of steps as first 2 ps of TR3


ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# ad_4 = ARPES_DATA[files[0]]
# ad_7 = ARPES_DATA[files[0]]
# ad_12 = ARPES_DATA[files[0]]

# ad_feb_1 = ARPES_DATA[files[0]]
# ad_feb_4 = ARPES_DATA[files[0]]

# Analysis Setup

In [ ]:
## Zero Delay, HOMO, EF ##

time_zero = 34.8225  # Bi2Se3

time_zero_feb = 37.96  # February

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400
homo_700 = homo + EF_700

In [ ]:
## Integrate over desired angular range ##

slice_dim = "x"
slice_val = 0
int_range = 20  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

xlim = None
ylim = None
x_bin = 1
y_bin = 1

In [ ]:
# ## Integrate over desired time range ##

# slice_dim = "z"
# slice_val = time_zero + tr_functions.ps_to_mm(0, time_zero)
# int_range = tr_functions.ps_to_mm(
#     0.5, time_zero
# )  # if this value is more that the integration range, my get_2D_slice function will just integrate over the max range.

# # xlim = (-20, 16.5)
# xlim = (-20, 15)
# ylim = None

# x_bin = 1
# y_bin = 1

In [ ]:
all_vals = []
for ad in [ad_feb_1, ad_feb_4, ad_12, ad_7, ad_4]:
    all_vals.append(
        tr_functions.slice_datacube(
            ad_dataclass=ad,
            slice_dim=slice_dim,
            slice_val=slice_val,
            int_range=int_range,
            xlim=xlim,
            ylim=(
                ad.energy[57],
                ad.energy[1007],
            ),  # get rid of zero padding on datasets
            x_bin=x_bin,
            y_bin=y_bin,
            norm_data=False,
            plot_data=False,
        )
    )

x_1, y_1, d_1 = all_vals[0]
x_4, y_4, d_4 = all_vals[1]
x_12, y_12, d_12 = all_vals[2]
x_7, y_7, d_7 = all_vals[3]
x_4_june, y_4_june, d_4_june = all_vals[4]

In [ ]:
## Adjust energy axis to be relative to HOMO and convert mm to ps ##

homo_zero = False
homo_zero = True

if homo_zero:
    # Feb TR4 and June #
    x4, y4, d4 = tr_functions.mm_to_ps(x_4, time_zero_feb), y_4 - homo_400, d_4
    x12, y12, d12 = tr_functions.mm_to_ps(x_12, time_zero), y_12 - homo_400, d_12
    x7, y7, d7 = tr_functions.mm_to_ps(x_7, time_zero), y_7 - homo_400, d_7
    x4_june, y4_june, d4_june = (
        tr_functions.mm_to_ps(x_4_june, time_zero),
        y_4_june - homo_400,
        d_4_june,
    )

    # Feb TR1 #
    x1, y1, d1 = tr_functions.mm_to_ps(x_1, time_zero_feb), y_1 - homo_700, d_1

# Analysis

In [ ]:
title = f"C<sub>60</sub> Pump Effect: {files[0]}"
# title = f"CT<sub>2</sub>"
title = f"C<sub>60</sub> Pump Effect"

yaxis_title = "E - E<sub>HOMO</sub> (eV)"
xaxis_title = "Delay (ps)"
# xaxis_title=f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"

In [ ]:
## Plot Data ##
# x_plot, y_plot, d_plot = x1, y1, d1
x_plot, y_plot, d_plot = x4, y4, d4
# x_plot, y_plot, d_plot = x12, y12, d12
# x_plot, y_plot, d_plot = x7, y7, d7
x_plot, y_plot, d_plot = x4_june, y4_june, d4_june

# # FFT angular dispersion data
# d_2d = filter_functions.fft2d_mask(d_2d, plot=False)

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=800,
    dtick_y=0.1,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

# fig.update_coloraxes(colorscale="greys", showscale=False)
# fig.update_coloraxes(cmin=0.0, cmax=0.8)

fig.show()

In [ ]:
T = 10.6  # measurement temp
k_B = 8.617333e-5  # eV/K

# title = "Waterfall Plot of G - K"
# x_plot, y_plot, z_plot = x12, y12, d12

title = "Waterfall Plot of M - K"
x_plot, y_plot, z_plot = x7, y7, d7

z_plot = analysis_functions.norm_data(z_plot)

spacing = 0.2

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot

slice_window = 10
steps = 20
new_axes = []
new_data = []
fits = []
fit_components = []
ylim = None
xlim = (np.min(x_edc), np.max(x_edc))
title = f"{title} ({slice_window} ps window)"

print(f"Minimum window size: {np.max(np.diff(x_edc))}")

fig = tr_functions.thesis_fig(
    title=title,
    yaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=400,
)

for i, xval in enumerate(np.linspace(xlim[0], xlim[1], steps)):
    x_window = (xval - slice_window / 2, xval + slice_window / 2)

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=x_window,
    )

    if len(col) == 1:
        continue

    fig.add_trace(
        go.Scatter(
            y=y_1d,
            x=i * spacing + col,
            name=f"{np.round(x_window[0], 1), np.round(x_window[1], 1)} ps",
            line=dict(color=colors[0]),
        )
    )
    new_axes.append(y_1d)
    new_data.append(col)

    x, data = y_1d, col
    offset_type = "constant"

    ## FD
    def fermi_dirac(x, center, theta, amp):
        arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
        return -amp / 2 * np.tanh(arg)

    ## Offset
    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    ## Fit for 2 peaks
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=2.2, sigma=0.1
    )  # 2.1
    gauss3 = fitting_functions.make_gaussian(
        num="C_", amplitude=1, center=2.55, sigma=0.1
    )  # 2.1

    ## Full model
    full_model = lm.models.Model(fermi_dirac) + gauss2 + gauss3 + offset

    params = full_model.make_params()

    if offset_type == "constant":
        # params = linear_params.copy()
        params = params
        params["b"].value = 0
        params["b"].vary = False

    params[
        "center"
    ].value = (
        2.0325  # TR12, determined by fitting from (0, 100) ps with a constant offset
    )
    # params["center"].vary = False
    params["theta"].value = k_B * (10.6)
    params["theta"].min = 0
    params["amp"].value = 1
    params["amp"].min = 0

    params["iB__center"].min = 2.0
    params["iB__sigma"].value = tr_functions.fwhm_to_sig(0.2)
    params["iB__sigma"].max = tr_functions.fwhm_to_sig(0.4)
    params["iC__center"].min = 2.4
    params["iC__sigma"].value = tr_functions.fwhm_to_sig(0.1)

    params["iC__sigma"].max = tr_functions.fwhm_to_sig(0.2)
    # params["iC__sigma"].max = tr_functions.fwhm_to_sig(0.4)

    fit = full_model.fit(data, x=x, params=params)

    fits.append(fit)
    fig.add_trace(
        go.Scatter(
            x=i * spacing + fit.eval(x=y_1d),
            y=y_1d,
            name="fit",
            line=dict(color="red"),
            opacity=0.5,
        )
    )

    components = fit.eval_components(x=y_1d)
    fit_components.append(components)


fig.show()

In [ ]:
xs = np.linspace(xlim[0], xlim[1], steps)
colors = pc.qualitative.D3

for param_name in ["amplitude", "height", "fwhm", "center"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(
        title=param_name,
        xaxis_title="delay (ps)",
        yaxis_title=param_name,
        font=dict(size=20),
    )

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("BC"):
        color = colors[i % len(colors) + 1]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
    fig.show()

In [ ]:
# xs = np.linspace(xlim[0], xlim[1], steps)
colors = pc.qualitative.D3

for param_name in ["height"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(
        title=param_name,
        xaxis_title="delay (ps)",
        yaxis_title=param_name,
        font=dict(size=20),
    )

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("B"):
        color = colors[i % len(colors) + 1]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        xs = np.linspace(xlim[0], xlim[1], len(ys))

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
fig.show()

In [ ]:
offset_type = "constant"
x = xs[np.where(xs < 26)]
ys = np.array(ys)
data = ys[np.where(xs < 26)]

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

gaussian = lm.models.ExponentialGaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

params["center"].value = 4
params["amplitude"].value = 1
params["sigma"].value = 1
# params["sigma"].value = tr_functions.fwhm_to_sig(0.5)
# params["sigma"].vary = False
params["gamma"].value = 1

fit = full_model.fit(data, x=x, params=params)

print(f'decay: {np.round(1 / fit.params["gamma"].value, 3)}')

fit.plot()

In [ ]:
fit

In [ ]:
param_name = "height"

fig = tr_functions.default_fig()
fig.update_layout(
    title=param_name,
    xaxis_title="delay (ps)",
    yaxis_title=param_name,
    font=dict(size=20),
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=data,
        error_y=dict(array=y_errs),
        name="data",
        line=dict(color=color),
        legendgroup=letter,
        legendgrouptitle_text=letter,
        mode="markers+lines",
    )
)

fig.add_trace(
    go.Scatter(
        x=x, y=fit.eval(x=x), name="fit", mode="lines", line=dict(color=colors[0])
    )
)

decay_error = (
    fit.params["gamma"].stderr
    / fit.params["gamma"].value
    * (1 / fit.params["gamma"].value)
)

fig.add_annotation(
    x=20,
    y=0.2,
    showarrow=False,
    text=f'Decay: {np.round(1 / fit.params["gamma"].value, 2)} +/- {decay_error:.2f} ps',
    font=dict(size=14),
    bgcolor="white",
    # opacity=1,
    bordercolor=color,
    borderwidth=2,
    # borderpad=4,
)


fig.show()

In [ ]:
tr_functions.sig_to_fwhm(fit.params["sigma"].value)

In [ ]:
x = xs[np.where(xs > 0)]
data = np.array(ys)[np.where(xs > 0)]

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)


def lowerstate(
    x, A_l, t0, tau_l, tau_u
):  # describes state that is decayed into and then continues to decay
    data = A_l * np.exp(-(x - t0) / tau_l) * (1 - np.exp(-(x - t0) / tau_u))
    return data


full_model = lm.models.Model(lowerstate) + offset

params = full_model.make_params()

params["A_l"].value = 1
params["t0"].value = 0
params["t0"].min = 0
params["tau_l"].value = 5
params["tau_l"].min = 0
params["tau_u"].value = 0.204
params["tau_u"].min = 0
params["tau_u"].vary = False

In [ ]:
offset_type = "constant"
# x = xs[np.where(xs < 26)]
# ys = np.array(ys)
# data = ys[np.where(xs < 26)]

x = xs[np.where(xs > 0)]
ys = np.array(ys)
data = ys[np.where(xs > 0)]

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)


def lowerstate(
    x, A_l, t0, tau_l, tau_u
):  # describes state that is decayed into and then continues to decay
    data = A_l * np.exp(-(x - t0) / tau_l) * (1 - np.exp(-(x - t0) / tau_u))
    return data


full_model = lm.models.Model(lowerstate) + offset

params = full_model.make_params()

params["A_l"].value = 1
params["t0"].value = 0
params["t0"].vary = False
params["tau_l"].value = 5
params["tau_l"].max = 100
params["tau_u"].value = 0.204  # M-K = 119; G-K = 124
# params["tau_u"].vary = False

# params["tau_u"].value = 1
# params["tau_u"].max = 1000

fit = full_model.fit(data, x=x, params=params)

print(f'decay upper: {np.round(fit.params["tau_u"].value, 3)}')
print(f'decay lower: {np.round(fit.params["tau_l"].value, 3)}')

fit.plot()

In [ ]:
param_name = "height"

## B (CT1)
color_data = colors[1]
color_fit = colors[0]


fig = tr_functions.default_fig()
fig.update_layout(
    title=param_name,
    xaxis_title="delay (ps)",
    yaxis_title=param_name,
    font=dict(size=20),
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=data,
        error_y=dict(array=y_errs),
        name="data",
        line=dict(color=color_data),
        legendgroup=letter,
        legendgrouptitle_text=letter,
        mode="markers+lines",
    )
)

fig.add_trace(
    go.Scatter(
        x=x, y=fit.eval(x=x), name="fit", mode="lines", line=dict(color=color_fit)
    )
)

for i, param in enumerate(["tau_u", "tau_l"]):
    fig.add_annotation(
        x=20,
        y=0.18 - i / 100,
        # y=0.2,
        showarrow=False,
        text=f"Decay: {np.round(fit.params[param].value, 2)} +/- {fit.params[param].stderr:.2f} ps",
        # text=f"Decay: {np.round(fit.params[param].value, 2)}",
        font=dict(size=14),
        bgcolor="white",
        # opacity=1,
        bordercolor=color_data,
        borderwidth=2,
        # borderpad=4,
    )

# fig.update_xaxes(range=(-2, 28))

fig.show()

# February

In [ ]:
title = "Waterfall Plot of G"  # TR1 Feb
x_plot, y_plot, z_plot = x1, y1, d1

z_plot = analysis_functions.norm_data(z_plot)

spacing = 0.2

if np.min(x_plot) > 0:
    x_edc = tr_functions.mm_to_ps(x_plot, time_zero)
else:
    x_edc = x_plot

slice_window = 10
steps = 30
new_axes = []
new_data = []
fits = []
fit_components = []
ylim = None
xlim = (np.min(x_edc), np.max(x_edc))
title = f"{title} ({slice_window} ps window)"


# xlim = (np.min(x_edc), 25)


print(f"Minimum window size: {np.max(np.diff(x_edc))}")

fig = tr_functions.thesis_fig(
    title=title,
    yaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    height=400,
)

for i, xval in enumerate(np.linspace(xlim[0], xlim[1], steps)):
    x_window = (xval - slice_window / 2, xval + slice_window / 2)

    y_1d, col = tr_functions.get_1d_x_slice(
        x=x_edc,
        y=y_plot,
        data=z_plot,
        ylims=ylim,
        x_range=x_window,
    )

    if len(col) == 1:
        continue

    fig.add_trace(
        go.Scatter(
            y=y_1d,
            x=i * spacing + col,
            name=f"{np.round(x_window[0], 1), np.round(x_window[1], 1)} ps",
            line=dict(color=colors[0]),
        )
    )
    new_axes.append(y_1d)
    new_data.append(col)

    x, data = y_1d, col
    offset_type = "constant"

    ## FD
    def fermi_dirac(x, center, theta, amp):
        arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
        return -amp / 2 * np.tanh(arg)

    ## Offset
    c = np.mean(data)
    b = (data[-1] - data[0]) / (x[-1] - x[0])
    a = 0

    offset = fitting_functions.offset_model(offset_type, a, b, c)

    # Fit for 2 peaks
    gauss1 = fitting_functions.make_gaussian(
        num="A_", amplitude=1, center=1.8, sigma=0.1
    )  # 1.86
    gauss2 = fitting_functions.make_gaussian(
        num="B_", amplitude=1, center=2.1, sigma=0.1
    )  # 2.1

    ## Full model
    full_model = lm.models.Model(fermi_dirac) + gauss1 + gauss2 + offset

    params = full_model.make_params()

    if offset_type == "constant":
        # params = linear_params.copy()
        params = params
        params["b"].value = 0
        params["b"].vary = False

    params[
        "center"
    ].value = 2.013  # determined by fitting from (0, 100) ps with a constant offset
    # params["center"].vary = False
    # # params["c"].value = 0  # determined by fitting from (0, 100) ps with a constant offset
    # # params["c"].vary = False
    params["theta"].value = k_B * (10.6)
    params["theta"].min = 0
    params["amp"].value = 1
    params["amp"].min = 0

    params["iA__sigma"].value = tr_functions.fwhm_to_sig(0.2)
    # params["iA__sigma"].vary = False
    params["iB__sigma"].value = tr_functions.fwhm_to_sig(0.1)
    params["iB__sigma"].max = tr_functions.fwhm_to_sig(0.2)

    fit = full_model.fit(data, x=x, params=params)
    fits.append(fit)
    fig.add_trace(
        go.Scatter(
            x=i * spacing + fit.eval(x=y_1d),
            y=y_1d,
            name="fit",
            line=dict(color="red"),
            opacity=0.5,
        )
    )

    components = fit.eval_components(x=y_1d)
    fit_components.append(components)

fig.show()

In [ ]:
xs = np.linspace(xlim[0], xlim[1], steps)
colors = pc.qualitative.D3

for param_name in ["amplitude", "height", "fwhm", "center"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(
        title=param_name,
        xaxis_title="delay (ps)",
        yaxis_title=param_name,
        font=dict(size=20),
    )

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("AB"):
        color = colors[i % len(colors)]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=xs,
        #         y=y_inits,
        #         name="inits",
        #         line=dict(color=color, dash="dash"),
        #         opacity=0.6,
        #         legendgroup=letter,
        #     )
        # )
    fig.show()

In [ ]:
# xs = np.linspace(xlim[0], xlim[1], steps)
colors = pc.qualitative.D3

for param_name in ["height"]:
    # Fig for each fitting parameter
    fig = tr_functions.default_fig()
    fig.update_layout(
        title=param_name,
        xaxis_title="delay (ps)",
        yaxis_title=param_name,
        font=dict(size=20),
    )

    # For each peak individually (one trace per peak)
    for i, letter in enumerate("B"):
        # for i, letter in enumerate("A"):
        color = colors[i % len(colors) + 1]
        # color = colors[i % len(colors) + 0]
        ys = []
        y_errs = []
        y_inits = []
        for fit in fits:
            par = fit.params[f"i{letter}__{param_name}"]
            ys.append(par.value)
            y_errs.append(par.stderr)
            y_inits.append(par.init_value)

        xs = np.linspace(xlim[0], xlim[1], len(ys))

        fig.add_trace(
            go.Scatter(
                x=xs,
                y=ys,
                error_y=dict(array=y_errs),
                name="value",
                line=dict(color=color),
                legendgroup=letter,
                legendgrouptitle_text=letter,
                mode="markers+lines",
            )
        )
fig.show()

In [ ]:
offset_type = "constant"
x = xs
data = np.array(ys)

# ## For comparison with June data region:
# x = xs[np.where(xs < 26)]
# ys = np.array(ys)
# data = ys[np.where(xs < 26)]

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

gaussian = lm.models.ExponentialGaussianModel()

full_model = gaussian + offset

params = full_model.make_params()

params["center"].value = 4
params["amplitude"].value = 0
params["sigma"].value = 2.6
# params["sigma"].vary = False
params["gamma"].value = 1

# ## xs < 26 params
# params["center"].value = 1
# params["amplitude"].value = 1
# params["sigma"].value = 1
# params["gamma"].value = 0.05

fit = full_model.fit(data, x=x, params=params)

print(
    f'decay: {np.round(1 / fit.params["gamma"].value, 3)} +/- {np.round(1 / fit.params["gamma"].stderr, 3)}'
)

fit.plot()

In [ ]:
fit

In [ ]:
param_name = "height"

## B (CT1)
color_data = colors[1]
color_fit = colors[0]

## A (S1)
# color_data = colors[0]
# color_fit = colors[4]

fig = tr_functions.default_fig()
fig.update_layout(
    title=param_name,
    xaxis_title="delay (ps)",
    yaxis_title=param_name,
    font=dict(size=20),
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=data,
        error_y=dict(array=y_errs),
        name="data",
        line=dict(color=color_data),
        legendgroup=letter,
        legendgrouptitle_text=letter,
        mode="markers+lines",
    )
)

fig.add_trace(
    go.Scatter(
        x=x, y=fit.eval(x=x), name="fit", mode="lines", line=dict(color=color_fit)
    )
)

decay_error = (
    fit.params["gamma"].stderr
    / fit.params["gamma"].value
    * (1 / fit.params["gamma"].value)
)

fig.add_annotation(
    x=80,
    y=0.08,
    # y=0.2,
    showarrow=False,
    text=f'Decay: {np.round(1 / fit.params["gamma"].value, 2)} +/- {decay_error:.2f} ps',
    font=dict(size=14),
    bgcolor="white",
    # opacity=1,
    bordercolor=color_data,
    borderwidth=2,
    # borderpad=4,
)

# fig.update_xaxes(range=(-2, 31))

fig.show()

In [ ]:
offset_type = "constant"
x = xs
data = np.array(ys)

x = xs[np.where(xs > 0)]
data = np.array(ys)[np.where(xs > 0)]

# x = x[np.where(x < 30)]
# data = data[np.where(x < 30)]

c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)


def lowerstate(
    x, A_l, t0, tau_l, tau_u
):  # describes state that is decayed into and then continues to decay
    data = A_l * np.exp(-(x - t0) / tau_l) * (1 - np.exp(-(x - t0) / tau_u))
    return data


full_model = lm.models.Model(lowerstate) + offset

params = full_model.make_params()

params["A_l"].value = 1
params["t0"].value = 0
params["t0"].min = 0
params["tau_l"].value = 5
params["tau_l"].min = 0
params["tau_u"].value = 0.204
params["tau_u"].min = 0
params["tau_u"].vary = False

fit = full_model.fit(data, x=x, params=params)

print(f'decay upper: {np.round(fit.params["tau_u"].value, 3)}')
print(f'decay lower: {np.round(fit.params["tau_l"].value, 3)}')

fit.plot()